In [ ]:
# %% [markdown]
"""
# **Depth Map Generator with Progress Monitoring**
* Real-time progress tracking for every batch  
* GPU memory monitoring  
* Error handling with retries
"""
# %%
import os
import time
import torch
import psutil
import humanize
from pathlib import Path
from tqdm.auto import tqdm
import torch.utils.data

# %% [markdown]
"""
## **1. Enhanced Progress Tracker**
"""
# %%
class ProgressTracker:
    def __init__(self, total_images):
        self.start_time = time.time()
        self.total_images = total_images
        self.completed = 0
        self.failed = 0
        self.last_update = 0
        
    def update(self, batch_size, failed=0):
        self.completed += batch_size - failed
        self.failed += failed
        
        # Update stats every 2 seconds
        if time.time() - self.last_update > 2 or self.completed == self.total_images:
            self._print_stats()
            self.last_update = time.time()
    
    def _print_stats(self):
        elapsed = time.time() - self.start_time
        speed = self.completed / max(elapsed, 1e-6)
        remaining = (self.total_images - self.completed) / max(speed, 1e-6)
        
        # GPU memory stats
        gpu_mem = humanize.naturalsize(torch.cuda.memory_allocated())
        gpu_max = humanize.naturalsize(torch.cuda.max_memory_allocated())
        
        print(
            f"\n📊 Progress: {self.completed}/{self.total_images} "
            f"({self.completed/self.total_images:.1%}) | "
            f"Speed: {speed:.1f} img/s | "
            f"ETA: {remaining:.0f}s\n"
            f"💾 GPU Mem: {gpu_mem} (Peak: {gpu_max}) | "
            f"Failures: {self.failed}"
        )

# %% [markdown]
"""
## **2. Enhanced Depth Generator**
"""
# %%
def generate_depth_maps(input_folder, output_folder, batch_size=16, max_retries=2):
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    # Get all image files
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
        image_paths.extend(list(input_folder.rglob(ext)))
    total_images = len(image_paths)
    
    # Initialize tracker
    tracker = ProgressTracker(total_images)
    print(f"🎯 Starting processing of {total_images} images in batches of {batch_size}")
    
    # Dataloader with error handling
    class SafeDataset(torch.utils.data.Dataset):
        def __init__(self, paths):
            self.paths = paths
            
        def __getitem__(self, idx):
            for _ in range(max_retries):
                try:
                    img = cv2.imread(str(self.paths[idx]))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    return torch.from_numpy(img).permute(2,0,1).float() / 255.0, str(self.paths[idx])
                except Exception as e:
                    print(f"⚠️ Error reading {self.paths[idx]}: {str(e)}")
                    time.sleep(1)
            return None, None
    
    dataset = SafeDataset(image_paths)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=4,
        collate_fn=lambda x: [item for item in x if item[0] is not None]
    )
    
    # Processing loop
    with torch.inference_mode(), torch.cuda.amp.autocast():
        for batch in tqdm(loader, desc="Total progress", unit="batch"):
            try:
                # Move data to GPU
                images = [item[0] for item in batch]
                paths = [item[1] for item in batch]
                batch_tensor = torch.stack(images).to('cuda').half()
                
                # Predict depth
                depth_outputs = model(batch_tensor)
                
                # Save results
                failed_in_batch = 0
                for i, depth in enumerate(depth_outputs):
                    try:
                        depth_map = process_depth(depth)
                        rel_path = Path(paths[i]).relative_to(input_folder)
                        save_path = output_folder / rel_path.with_suffix('.png')
                        save_path.parent.mkdir(parents=True, exist_ok=True)
                        cv2.imwrite(str(save_path), depth_map)
                    except Exception as e:
                        print(f"⚠️ Failed to save {paths[i]}: {str(e)}")
                        failed_in_batch += 1
                
                # Update progress
                tracker.update(len(batch), failed_in_batch)
                
            except RuntimeError as e:
                if 'CUDA out of memory' in str(e):
                    print("💥 OOM detected, reducing batch size...")
                    batch_size = max(1, batch_size // 2)
                    loader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size,
                        num_workers=2
                    )
                else:
                    print(f"⚠️ Batch failed: {str(e)}")
                    tracker.update(len(batch), len(batch))
    
    print(f"\n✅ Completed! Processed {tracker.completed} images")
    print(f"   Failures: {tracker.failed}")
    print(f"   Peak GPU memory: {humanize.naturalsize(torch.cuda.max_memory_allocated())}")
    print(f"   Total time: {time.time() - tracker.start_time:.1f}s")

# %% [markdown]
"""
## **3. Run with Full Monitoring**
"""
# %%
# Configuration
INPUT_ROOT = "/content/input"
OUTPUT_ROOT = "/content/depth_maps"
BATCH_SIZE = 32  # Start high, auto-reduces on OOM

# Initialize model
model = torch.hub.load('intel-isl/MiDaS', 'DPT_Large').eval().half().to('cuda')

# Start processing
generate_depth_maps(INPUT_ROOT, OUTPUT_ROOT, batch_size=BATCH_SIZE)